<a href="https://colab.research.google.com/github/chadantomar/ML/blob/main/readVideoContent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pytube

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.6 MB/s eta 0:00:00


In [2]:
from pytube import YouTube

In [10]:
youtube = YouTube("https://www.youtube.com/watch?v=v4t0E3S1N1k")

# Get the highest resolution stream available
video_stream = youtube.streams.get_highest_resolution()

# Download the video
video_stream.download(output_path="/content/drive/MyDrive/Deep learing/Tensorflow/data/youtube.mp4")
print("Download complete!")

Download complete!


In [4]:
!pip install torch torchvision

In [12]:
torch.cuda.is_available()

True

In [21]:
model = models.video.r3d_18(pretrained=True)


In [22]:
dir(model)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_backward_hooks',
 '_backward_pre_hooks',
 '_buffers',
 '_call_impl',
 '_compiled_call_impl',
 '_forward_hooks',
 '_forward_hooks_always_called',
 '_forward_hooks_with_kwargs',
 '_forward_pre_hooks',
 '_forward_pre_hooks_with_kwargs',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_is_full_backward_hook',
 '_load_from_state_dict',
 '_load_state_dict_post_hooks',
 '_load_state_dict_pre_hooks',
 '_make_layer',
 '_maybe_warn_non_full_backward_hook',
 '_modules',
 '_named_members',
 '_non_persi

In [20]:
import torch
from torchvision import models, transforms
from torch.autograd import Variable
from torch.nn import functional as F
from PIL import Image
import cv2
import numpy as np

# Load a pre-trained 3D ResNet model
model = models.video.r3d_18(pretrained=True)

# Set the model to evaluation mode
model.eval()

# Get the expected number of channels from the model
expected_channels = model.stem[0].weight.shape[1]

# Define preprocessing transformations
preprocess = transforms.Compose([
    transforms.Resize((128, 171)),  # Resize frames
    transforms.CenterCrop((112, 112)),  # Crop to the input size expected by the model
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
])

# Function to extract features from a video
def extract_video_features(video_path):
    print("hello"+video_path)
    cap = cv2.VideoCapture(video_path)
    frames = []

    # Read video frames
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Resize and crop the frame
        frame_resized = cv2.resize(frame, (171, 128))
        frame_cropped = frame_resized[8:120, 30:142, :]

        # Convert the frame to PIL Image and apply preprocessing
        frame_pil = Image.fromarray(cv2.cvtColor(frame_cropped, cv2.COLOR_BGR2RGB))
        frame_tensor = preprocess(frame_pil)

        # Expand the number of channels to match the expected_channels
        current_channels = frame_tensor.shape[0]
        expected_channels = 3
        if current_channels != expected_channels:
            frame_tensor = frame_tensor.repeat(expected_channels // current_channels, 1, 1)

        frames.append(frame_tensor)

    cap.release()

    # Stack the frames along the time dimension
    frames_tensor = torch.stack(frames)

    # Add batch dimension
    frames_tensor = frames_tensor.unsqueeze(0)

    # Permute the dimensions dynamically
    num_dims = frames_tensor.dim()
    time_dim = next((i for i, dim in enumerate(frames_tensor.shape) if dim > 1), None)  # Find the first dimension with size > 1
    permuted_dims = [0] + list(range(1, time_dim)) + [time_dim] + list(range(time_dim + 1, num_dims))
    frames_tensor = frames_tensor.permute(*permuted_dims)

    # Adjust the number of channels if needed
    for name, module in model.named_children():

      if isinstance(module, torch.nn.Conv2d):
          expected_channels_model = module.in_channels
           current_channels = frames_tensor.shape[2]
          if current_channels != expected_channels_model:
              frames_tensor = frames_tensor.repeat(1, 1, expected_channels_model // current_channels, 1, 1)
          break  # Adjust this according to the model's first convolutional layer




    # Prepare input for the model
    input_var = Variable(frames_tensor)
    print("Input Tensor Shape:", input_var.shape)
    try:
        # Ensure the model is in evaluation mode
        model.eval()

        # Move model and input tensor to the same device if using GPU
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model.to(device)
        input_var = input_var.to(device)

        with torch.no_grad():
            output = model(input_var)

        print("Output Tensor Shape:", output.shape)
        video_vector = F.avg_pool3d(output, (output.size(-3), output.size(-2), output.size(-1)))
        #print("video_vector Shape:", video_vector.shape)
        video_vector = video_vector.squeeze(3).squeeze(2).squeeze(1)
        print(video_vector)
        return video_vector.numpy()

    except Exception as e:
        print("Error during model forward pass:", e)
    # Use the output features as the video vector

# Example usage
print("hello")
video_path = "/content/drive/MyDrive/Deep learing/Tensorflow/data/youtube.mp4/English in a Minute Grow Like a Weed.mp4"
video_vector = extract_video_features(video_path)
print("Video Vector Shape:", video_vector.shape)


hello
hello/content/drive/MyDrive/Deep learing/Tensorflow/data/youtube.mp4/English in a Minute Grow Like a Weed.mp4
Input Tensor Shape: torch.Size([1, 1799, 3, 112, 112])
Error during model forward pass: Given groups=1, weight of size [64, 3, 3, 7, 7], expected input[1, 1799, 3, 112, 112] to have 3 channels, but got 1799 channels instead


AttributeError: 'NoneType' object has no attribute 'shape'

In [44]:
import torch
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image
import cv2

# Load the pre-trained R3D-18 model
model = models.video.r3d_18(pretrained=True)

# Set the model to evaluation mode
model.eval()

# Path to your video file
video_path = "/content/drive/MyDrive/Deep learing/Tensorflow/data/youtube.mp4/English in a Minute Grow Like a Weed.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

# Placeholder for frames_list
frames_list = []

# Read frames from the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Resize and crop the frame
    frame_resized = cv2.resize(frame, (171, 128))
    frame_cropped = frame_resized[8:120, 30:142, :]

    # Convert the frame to PIL Image
    frame_pil = Image.fromarray(cv2.cvtColor(frame_cropped, cv2.COLOR_BGR2RGB))

    # Preprocess the frame
    frame_tensor = transforms.ToTensor()(frame_pil)

    # Add the preprocessed frame to frames_list
    frames_list.append(frame_tensor)

cap.release()

# Convert frames_list to torch.Tensor
frames_tensor = torch.stack(frames_list)
frames_tensor = frames_tensor.unsqueeze(0)  # Add batch dimension

# Move frames_tensor to the same device as the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
frames_tensor = frames_tensor.to(device)

conv1 = model.stem[0]

# Print information about the model's first convolutional layer
print("Model Conv1 Input Channels:", conv1.in_channels)

# Print information about the input data
print("Input Tensor Shape:", frames_tensor.shape)
print("Expected Channels for Conv1:", conv1.in_channels)

# Dynamically get the expected number of channels for the first convolutional layer (conv1)
expected_channels_model = conv1.in_channels

# Adjust the number of channels if needed
current_channels = frames_tensor.shape[2]
if current_channels != expected_channels_model:
    frames_tensor = frames_tensor[:, :expected_channels_model, :, :, :]


# Perform the forward pass
with torch.no_grad():
    output = model(frames_tensor)

# Use the output features as the video vector
video_vector = F.avg_pool3d(output, output.shape[-3:])
video_vector = video_vector.squeeze(-1).squeeze(-1).squeeze(-1)

print("Video Vector Shape:", video_vector.shape)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model Conv1 Input Channels: 3
Input Tensor Shape: torch.Size([1, 1799, 3, 112, 112])
Expected Channels for Conv1: 3


RuntimeError: Given groups=1, weight of size [64, 3, 3, 7, 7], expected input[1, 1799, 3, 112, 112] to have 3 channels, but got 1799 channels instead

In [43]:
 dir(conv1.in_channels)

['__abs__',
 '__add__',
 '__and__',
 '__bool__',
 '__ceil__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__divmod__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floor__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__le__',
 '__lshift__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdivmod__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rlshift__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__round__',
 '__rpow__',
 '__rrshift__',
 '__rshift__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__trunc__',
 '__xor__',
 'as_integer_ratio',
 'bit_count',
 'bit_length',
 'conjugate',
 'denominator',
 'from_bytes',
 'imag',
 'numerator',
 'real',
 'to_bytes